## Installing and Setting Up Ultralytics YOLOv8 for Object Detection

In [ ]:
# Install the Ultralytics library for YOLOv8
!pip install ultralytics

# Import necessary libraries
from IPython import display

# Clear the output in the notebook
display.clear_output()

# Import the ultralytics library
import ultralytics

# Run checks for the Ultralytics installation
ultralytics.checks()

Ultralytics YOLOv8.2.2 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 29.0/107.7 GB disk)


## Loading the YOLOv8 Model and Utilities for Image Display

In [ ]:
# Import the YOLO model from the ultralytics library
from ultralytics import YOLO

# Import necessary utilities from IPython for displaying images
from IPython.display import display, Image

In [ ]:
!yolo task=detect mode=predict model=yolov8s.pt conf=0.25 source=/content/drive/MyDrive/Tested_images/test_2 save=True

100% 21.5M/21.5M [00:00<00:00, 50.6MB/s]
Ultralytics YOLOv8.2.2 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8s summary (fused): 168 layers, 11156544 parameters, 0 gradients, 28.6 GFLOPs

Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/cfg/__init__.py", line 582, in entrypoint
    getattr(model, mode)(**overrides)  # default args from model
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/engine/model.py", line 452, in predict
    return self.predictor.predict_cli(source=source) if is_cli else self.predictor(source=source, stream=stream)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/engine/predictor.py", line 177, in predict_cli
    for _ in gen:  # noqa, running CLI inference without accumulating any outputs (do not modify)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 35, in gene

## Mounting Google Drive to Colab

In [ ]:
# Connecting Google Drive to Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Setting the Working Directory to Google Drive Location

In [ ]:
# Defining Drive path to save the dataset
%cd /content/drive/MyDrive/ML02

/content/drive/MyDrive/ML02


## Downloading Dataset from Roboflow

In [ ]:
# Install the Roboflow library
!pip install roboflow

# Import the Roboflow library
from roboflow import Roboflow

# Initialize the Roboflow client with API key
rf = Roboflow(api_key="wY4ih6xnHAB4UC22wlac")

# Get the project from your Roboflow workspace
project = rf.workspace("fishtypes").project("fishtypes")

# Get the specific version of the project
version = project.version(9)

# Download the dataset for the specified version in YOLOv8 format
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.2, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to fishtypes-9 in yolov8:: 100%|██████████| 13284/13284 [00:05<00:00, 2421.28it/s]


In [ ]:
# Doube Checking the File Path
!ls

drive  fishtypes-9


## Inspecting entries information in Dataset

In [ ]:
import os
from PIL import Image
from tabulate import tabulate

# Defining dataset paths in drive
dataset_paths = [
    "/content/fishtypes-9/test/images",
    "/content/fishtypes-9/train/images",
    "/content/fishtypes-9/valid/images"
]

# Function to get dataset information for a given path
def get_dataset_info(dataset_path):
    total_images = 0
    resolutions = set()
    image_formats = set()

    # Loop through all files in the directory
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if file.endswith(('.jpg', '.png', '.bmp')):
                image_path = os.path.join(root, file)
                image = Image.open(image_path)
                width, height = image.size
                resolutions.add((width, height))
                total_images += 1

                # Get image format
                image_format = image.format
                image_formats.add(image_format)

    return total_images, resolutions, image_formats

# Get total number of images in all folders
total_images_all_folders = sum(get_dataset_info(dataset_path)[0] for dataset_path in dataset_paths)

# Get dataset information for each folder
dataset_info = []

for dataset_path in dataset_paths:
    folder_name = os.path.basename(os.path.dirname(dataset_path))
    total_images, resolutions, image_formats = get_dataset_info(dataset_path)
    percentage = (total_images / total_images_all_folders) * 100 if total_images_all_folders > 0 else 0
    dataset_info.append((folder_name, total_images, f"{percentage:.2f}%", resolutions, image_formats))

# Print dataset information in a table
headers = ["Folder", "Total Images", "Percentage", "Resolutions", "Image Formats"]
print(tabulate(dataset_info, headers=headers, tablefmt="grid"))
print(f"Total images in the dataset: {total_images_all_folders}")

+----------+----------------+--------------+---------------+-----------------+
| Folder   |   Total Images | Percentage   | Resolutions   | Image Formats   |
+==========+================+==============+===============+=================+
| test     |            492 | 7.41%        | {(640, 640)}  | {'JPEG'}        |
+----------+----------------+--------------+---------------+-----------------+
| train    |           5160 | 77.76%       | {(640, 640)}  | {'JPEG'}        |
+----------+----------------+--------------+---------------+-----------------+
| valid    |            984 | 14.83%       | {(640, 640)}  | {'JPEG'}        |
+----------+----------------+--------------+---------------+-----------------+
Total images in the dataset: 6636


## Inspecting Classes information in Dataset From data.yaml file

In [ ]:
import os
import yaml
from tabulate import tabulate

# Path to the data.yaml file
data_yaml_path = '/content/fishtypes-9/data.yaml'

# Load the data.yaml file
with open(data_yaml_path, 'r') as file:
    data = yaml.safe_load(file)

# Get the list of classes
classes = data['names']

# Initialize a dictionary to store the count of data entries for each class
class_counts = {class_name: 0 for class_name in classes}

# Define the label folders
label_folders = ['test/labels', 'train/labels', 'valid/labels']

# Traverse through the label folders
for label_folder in label_folders:
    label_folder_path = os.path.join('/content/fishtypes-9', label_folder)

    # Iterate through each label file in the label folder
    for file_name in os.listdir(label_folder_path):
        file_path = os.path.join(label_folder_path, file_name)

        # Open and read the label file
        with open(file_path, 'r') as label_file:
            lines = label_file.readlines()

            # Extract class indices from the label file and update class counts
            for line in lines:
                class_index = int(line.split()[0])
                class_name = classes[class_index]
                class_counts[class_name] += 1

# Calculate total count of classes
total_classes = len(classes)

# Prepare data for tabulate
table_data = [(class_name, count) for class_name, count in class_counts.items()]

# Append total class count to table data
table_data.append(("Total Classes", total_classes))

# Print the summary as a table
print(tabulate(table_data, headers=["Class", "Entries"], tablefmt="grid"))


+----------------------------+-----------+
| Class                      |   Entries |
+============================+===========+
| fish_Bangus 1              |       268 |
+----------------------------+-----------+
| fish_Big Head Carp         |       328 |
+----------------------------+-----------+
| fish_Black Spotted Barb    |       306 |
+----------------------------+-----------+
| fish_Catfish               |       446 |
+----------------------------+-----------+
| fish_Climbing Perch        |       215 |
+----------------------------+-----------+
| fish_Fourfinger Threadfin  |       319 |
+----------------------------+-----------+
| fish_Freshwater Eel        |       364 |
+----------------------------+-----------+
| fish_Glass Perchlet        |       655 |
+----------------------------+-----------+
| fish_Goby                  |       899 |
+----------------------------+-----------+
| fish_Gourami               |       426 |
+----------------------------+-----------+
| fish_Gras

## Training the YOLOv8 Object Detection Model

In [ ]:
# Change the current working directory to the specified path
%cd /content/fishtypes-9

# Run the YOLOv8 object detection training command
!yolo task=detect mode=train model=yolov8s.pt data=data.yaml epochs=40 imgsz=640 plots=True lr0=0.01 lrf=0.01

/content/fishtypes-9
100% 21.5M/21.5M [00:00<00:00, 212MB/s]

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.yaml, data=data.yaml, epochs=40, patience=50, batch=16, imgsz=640, save=True, cache=False, device=, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, 

### Transfering Final data to Google Drive

In [ ]:
# Source and destination paths
source_path = '/content/fishtypes-9/yolov8s.pt'  # Source directory containing the folder to be copied
destination_path = '/content/drive/MyDrive'  # Destination directory where the folder will be copied

# Copy the folder from source to destination
!cp -r {source_path} {destination_path}  # Copying the folder recursively (-r) from source to destination


In [ ]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/runs/detect/train/weights/best.pt conf=0.25 source=/content/drive/MyDrive/Tested_images save=True

2024-04-22 16:41:35.531536: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-22 16:41:35.531949: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-22 16:41:35.534056: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-22 16:41:37.554231: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU
Model summary (fused): 168 layers, 11133324 parameters, 0 gradients, 28.5 GFLOPs
image 1/7 /content/drive/MyDrive/Tested_images/10053681323_87b19c1985.jpg: 384x640 1 fish_Black S

In [ ]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/runs/detect/train/weights/best.pt conf=0.25 source=/content/drive/MyDrive/Tested_images/test_2 save=True

2024-04-22 17:10:19.818079: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-22 17:10:19.818154: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-22 17:10:19.820249: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-22 17:10:21.737687: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU
Model summary (fused): 168 layers, 11133324 parameters, 0 gradients, 28.5 GFLOPs
image 1/17 /content/drive/MyDrive/Tested_images/test_2/1000_F_231001810_nuUCxZiE3QGXVhDtfxN3Ju3dS